# Lesson 3: Building an Agent Reasoning Loop

In [72]:
from custom_utils.helper import get_openai_api_key, print_response

OPENAI_API_KEY = get_openai_api_key()

In [73]:
import nest_asyncio

nest_asyncio.apply()

## Setup the Query Tools

In [74]:
from custom_utils.utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("../resources/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [75]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [76]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)
print_response(response.response)

In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")
print_response(response.response)

## Lower-Level: Debuggability and Control

In [82]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [83]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print_response(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

In [89]:
response = agent.finalize_response(task.task_id)

In [ ]:
print_response(str(response))